In [ ]:
!pip install google-generativeai sentence-transformers requests beautifulsoup4 faiss-cpu pandas tldextract



In [ ]:
import pandas as pd
import requests
import requests.exceptions
from bs4 import BeautifulSoup
import google.generativeai as genai
import google.api_core.exceptions
from sentence_transformers import SentenceTransformer
import tldextract
import numpy as np
import faiss
import time
import json
import re

In [ ]:
# Load your CSV
df = pd.read_csv('/content/assignment_uno_southAfrica.csv')

In [ ]:
# --- Configuration ---
API_KEY = ""  # Replace with your actual Gemini API key
genai.configure(api_key=API_KEY)

gem_model = genai.GenerativeModel("gemini-2.5-flash")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# ----------------------------
# HELPER FUNCTIONS
# ----------------------------
def clean_url(url):
    if not isinstance(url, str) or len(url) < 5:
        return None
    if not url.startswith("http"):
        url = "https://" + url
    return url

def scrape_site(url):
    """Scrape website and return comprehensive text content"""
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")

        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer"]):
            script.decompose()

        # Get text
        text = soup.get_text(" ", strip=True)

        # Try to get more structured info
        meta_desc = soup.find("meta", {"name": "description"})
        if meta_desc and meta_desc.get("content"):
            text = meta_desc.get("content") + " " + text

        return text[:15000]  # Limit to ~15k chars to avoid huge contexts
    except requests.exceptions.RequestException as e:
        print(f"Scrape error for {url}: {e}")
        return ""

def chunk_text(text, size=500):
    """Create overlapping chunks for better context"""
    words = text.split()
    chunks = []
    for i in range(0, len(words), size):
        chunk = " ".join(words[i:i+size])
        if chunk:
            chunks.append(chunk)
    return chunks

def query_llm(prompt, max_retries=5):
    """Send prompt to Gemini with retry + exponential backoff"""
    wait = 5
    for attempt in range(max_retries):
        try:
            output = gem_model.generate_content(prompt).text
            return output
        except google.api_core.exceptions.ResourceExhausted as e:
            print(f"Rate limit hit. Waiting {wait}s before retry {attempt+1}...")
            time.sleep(wait)
            wait *= 2
        except Exception as e:
            if "rate limit" in str(e).lower() or "429" in str(e):
                print(f"Rate limit hit. Waiting {wait}s before retry {attempt+1}...")
                time.sleep(wait)
                wait *= 2
            else:
                print(f"LLM query failed: {e}")
                raise
    print("Max retries exceeded.")
    return ""

def extract_relevant_context(text, company_name, num_chunks=5):
    """Use semantic search to find most relevant parts of website"""
    if not text or len(text) < 50:
        return "No content available"

    chunks = chunk_text(text, size=400)
    if not chunks:
        return "No content available"

    # Create embeddings
    embeddings = embedder.encode(chunks).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    # Search for relevant chunks using multiple queries
    queries = [
        company_name,
        f"{company_name} loan interest rate terms",
        f"{company_name} financing credit requirements",
        f"{company_name} headquarters employees founded"
    ]

    relevant_chunks = set()
    for query in queries:
        qvec = embedder.encode([query]).astype("float32")
        _, idxs = index.search(qvec, min(2, len(chunks)))
        for idx in idxs[0]:
            if idx < len(chunks):
                relevant_chunks.add(idx)

    # Combine relevant chunks
    context = " ".join([chunks[i] for i in sorted(relevant_chunks)[:num_chunks]])
    return context[:5000]  # Limit context size

In [ ]:
# ----------------------------
# MAIN PROCESSING LOOP
# ----------------------------
batch_size = 1  # Process one at a time for better accuracy
results = []

# Define all columns we need to populate
target_columns = [
    'headquarters_city', 'founded_year', 'employee_count', 'annual_revenue_usd',
    'financing_volume_usd', 'market_share_percentage', 'business_model',
    'financing_types', 'credit_requirements', 'avg_approval_rate',
    'avg_interest_rate', 'max_financing_amount_usd', 'avg_term_months', 'status'
]

print("Starting processing...")
for idx, row in df.iterrows():
    print(f"\n{'='*60}")
    print(f"Processing {idx + 1}/{len(df)}: {row['name']}")
    print('='*60)

    name = row["name"]
    legal_name = row.get("legal_name", name)
    url = clean_url(row["website"])

    # Scrape website
    text = scrape_site(url) if url else ""
    context = extract_relevant_context(text, name)

    # Create comprehensive prompt
    prompt = f"""You are a financial data analyst. Extract ALL available information about the following company from the provided context.

Company Name: {name}
Legal Name: {legal_name}
Website: {url}

Context from website:
{context}

Please extract and provide the following information in a VALID JSON object format. For each field:
- If you find the information, provide the exact value
- If you can reasonably infer from context, provide your best estimate with a note
- If completely unknown, use null

Required fields:
{{
  "company_name": "{name}",
  "headquarters_city": "City, Country",
  "founded_year": 2020,
  "employee_count": 100,
  "annual_revenue_usd": 1000000,
  "financing_volume_usd": 5000000,
  "market_share_percentage": 5.5,
  "business_model": "Brief description of how they make money",
  "financing_types": "List types: e.g., 'Device financing, BNPL, Personal loans'",
  "credit_requirements": "Description of credit requirements or eligibility",
  "avg_approval_rate": 75.5,
  "avg_interest_rate": 12.5,
  "max_financing_amount_usd": 2000,
  "avg_term_months": 12,
  "status": "Active/Inactive/Unknown"
}}

IMPORTANT:
1. For South African companies, provide amounts in USD (convert from ZAR if needed, use ~0.055 conversion rate)
2. For interest rates, provide annual percentage rate (APR) as a number
3. For employee_count, if you see ranges like "50-200", use the midpoint (125)
4. For market_share_percentage, estimate based on company size/presence if exact data unavailable
5. Be specific and detailed for business_model and financing_types
6. Return ONLY the JSON object, nothing else"""

    try:
        output_text = query_llm(prompt)

        # Parse JSON
        cleaned = re.sub(r"```(?:json)?", "", output_text).strip()
        start_idx = cleaned.find('{')
        end_idx = cleaned.rfind('}')

        if start_idx != -1 and end_idx != -1:
            json_str = cleaned[start_idx : end_idx + 1]
            parsed = json.loads(json_str)

            # Ensure all required fields exist
            result = {
                "id": row["id"],
                "name": name,
                "legal_name": legal_name,
                "website": row["website"]
            }

            # Add extracted fields
            for col in target_columns:
                result[col] = parsed.get(col, None)

            results.append(result)
            print(f"✅ Successfully extracted data for {name}")

        else:
            raise ValueError("No JSON object found")

    except Exception as e:
        print(f"❌ Failed to process {name}: {e}")
        print(f"Raw output: {output_text[:300]}...")

        # Create record with nulls
        result = {
            "id": row["id"],
            "name": name,
            "legal_name": legal_name,
            "website": row["website"]
        }
        for col in target_columns:
            result[col] = None
        result["error"] = str(e)
        results.append(result)

    # Rate limiting - be gentle with free API
    time.sleep(8)

Starting processing...

Processing 1/54: Cellucity
✅ Successfully extracted data for Cellucity

Processing 2/54: PayJoy South Africa
✅ Successfully extracted data for PayJoy South Africa

Processing 3/54: Vodacom South Africa
✅ Successfully extracted data for Vodacom South Africa

Processing 4/54: FoneYam
Scrape error for https://foneyam.com: HTTPSConnectionPool(host='foneyam.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb04f36dcd0>: Failed to resolve 'foneyam.com' ([Errno -5] No address associated with hostname)"))
✅ Successfully extracted data for FoneYam

Processing 5/54: Standard Bank Connect
Scrape error for https://connect.standardbank.co.za: 403 Client Error: Forbidden for url: https://connect.standardbank.co.za/
✅ Successfully extracted data for Standard Bank Connect

Processing 6/54: MTN South Africa
✅ Successfully extracted data for MTN South Africa

Processing 7/54: PayJustNow
✅ Successfull

In [ ]:
# Create final dataframe
results_df = pd.DataFrame(results)

# Reorder columns to match original
all_columns = ['id', 'name', 'legal_name', 'website'] + target_columns + ['created_at', 'updated_at']
for col in all_columns:
    if col not in results_df.columns:
        results_df[col] = None

results_df = results_df[all_columns]

print("\n" + "="*60)
print(f"Processing complete! Total records: {len(results_df)}")
print("="*60)
print("\nSample of extracted data:")
print(results_df[['name', 'headquarters_city', 'founded_year', 'avg_interest_rate', 'financing_types']].head())

# Save results
results_df.to_csv('populated_results.csv', index=False)
print("\n✅ Results saved to 'populated_results.csv'")

# Show summary statistics
print("\n" + "="*60)
print("DATA COMPLETENESS SUMMARY:")
print("="*60)
for col in target_columns:
    non_null = results_df[col].notna().sum()
    percentage = (non_null / len(results_df)) * 100
    print(f"{col:30s}: {non_null:3d}/{len(results_df)} ({percentage:5.1f}%)")


Processing complete! Total records: 54

Sample of extracted data:
                    name headquarters_city  founded_year  avg_interest_rate  \
0              Cellucity              None           NaN                NaN   
1    PayJoy South Africa              None           NaN                NaN   
2   Vodacom South Africa              None           NaN                NaN   
3                FoneYam              None           NaN                NaN   
4  Standard Bank Connect              None           NaN                NaN   

                                     financing_types  
0  Vodacom Contracts, LayUp (Buy Now, Pay Later),...  
1  Device financing (BNPL for smartphones), Perso...  
2              Device financing (via contract plans)  
3                                               None  
4                                               None  

✅ Results saved to 'populated_results.csv'

DATA COMPLETENESS SUMMARY:
headquarters_city             :  12/54 ( 22.2%)
founded_

In [ ]:
df2 = pd.read_csv('/content/populated_results.csv')
df2.sample(10)

,id,name,legal_name,website,headquarters_city,founded_year,employee_count,annual_revenue_usd,financing_volume_usd,market_share_percentage,business_model,financing_types,credit_requirements,avg_approval_rate,avg_interest_rate,max_financing_amount_usd,avg_term_months,status,created_at,updated_at
0,338f1999-1a3c-43e2-8d31-f6076191fecb,Cellucity,Cellucity (Pty) Ltd,https://cellucity.co.za,NaN,NaN,NaN,NaN,NaN,NaN,Cellucity operates as a multi-channel retailer...,"Vodacom Contracts, LayUp (Buy Now, Pay Later),...","For contract deals, eligibility is subject to ...",NaN,NaN,1485.0,36.0,Active,NaN,NaN
13,81bb3202-14f8-470f-8f8a-fb1bdc4aff62,Mobicred,Mobicred (Pty) Ltd,https://www.mobicred.co.za,NaN,NaN,NaN,NaN,NaN,NaN,Mobicred provides a revolving credit facility ...,Revolving Credit,Applicants must have a valid South African ID ...,NaN,NaN,NaN,NaN,Active,NaN,NaN
41,03f9d6ab-27d3-46aa-92ec-22493f11e410,Discovery Bank Limited,Discovery Bank Limited,https://www.discovery.co.za,NaN,NaN,NaN,NaN,NaN,NaN,"Provides retail banking services, including ba...",NaN,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
14,8e7b7b8d-a4cc-4451-9851-8beaddbc02f7,Mr Price Mobile,Mr Price Mobile (Pty) Ltd,https://www.mrpmobile.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,cd312ec9-897b-4900-b2b4-bd76f0cc44e0,African Bank Limited,African Bank Limited,https://www.africanbank.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,7f2c0539-8514-4478-b6aa-a635c942dc3d,Float,Float (Pty) Ltd,https://www.float.co.za,NaN,NaN,NaN,NaN,NaN,NaN,"Float facilitates interest-free, fee-free inst...",Installment payments (leveraging existing cred...,"Requires an existing, valid Visa or Mastercard...",NaN,0.0,825.0,NaN,Active,NaN,NaN
42,2a6c7a7c-a0bd-43ab-bd5c-f5a5bd67081e,Melon Mobile,Melon Mobile (Pty) Ltd,https://www.melonmobile.co.za,NaN,NaN,NaN,NaN,NaN,NaN,Melon Mobile operates as a Mobile Virtual Netw...,Device financing (installment plans),NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
34,9cbbb8e0-0155-42dc-9ca2-2d542140398e,Yoco,Yoco Technologies (Pty) Ltd,https://www.yoco.co.za,NaN,NaN,NaN,NaN,NaN,NaN,Yoco provides payment processing solutions and...,NaN,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
33,704c72e2-e847-4b04-85c3-754c6ebeded1,TFG Connect MVNO,TFG Connect (MVNO),https://www.tfg.co.za/connect,NaN,NaN,NaN,NaN,NaN,NaN,"As a Mobile Virtual Network Operator (MVNO), T...",NaN,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN
21,7a1fc133-3460-402c-ae51-010db67ac955,FirstRand Bank Limited (FNB),FirstRand Bank Limited,https://www.fnb.co.za,"Sandton, South Africa",1929.0,NaN,NaN,NaN,NaN,"FNB operates as a comprehensive retail, commer...","['Cash advances', 'Overdrafts', 'Debtor financ...",NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN


In [ ]:
df2.head()

,id,name,legal_name,website,headquarters_city,founded_year,employee_count,annual_revenue_usd,financing_volume_usd,market_share_percentage,business_model,financing_types,credit_requirements,avg_approval_rate,avg_interest_rate,max_financing_amount_usd,avg_term_months,status,created_at,updated_at
0,338f1999-1a3c-43e2-8d31-f6076191fecb,Cellucity,Cellucity (Pty) Ltd,https://cellucity.co.za,NaN,NaN,NaN,NaN,NaN,NaN,Cellucity operates as a multi-channel retailer...,"Vodacom Contracts, LayUp (Buy Now, Pay Later),...","For contract deals, eligibility is subject to ...",NaN,NaN,1485.00,36.0,Active,NaN,NaN
1,5c0101bc-e330-4cbd-968f-712e83458e6d,PayJoy South Africa,PayJoy South Africa (Pty) Ltd,https://www.payjoy.com/za,NaN,NaN,NaN,NaN,NaN,NaN,PayJoy South Africa provides credit for acquir...,"Device financing (BNPL for smartphones), Perso...",Requires a valid South African ID or Passport....,NaN,NaN,NaN,6.0,Active,NaN,NaN
2,88487866-2191-406c-b0b7-5a07d81def4b,Vodacom South Africa,Vodacom (Pty) Ltd,https://www.vodacom.co.za,NaN,NaN,NaN,NaN,NaN,NaN,A telecommunications company providing mobile ...,Device financing (via contract plans),Likely required for long-term device contract ...,NaN,NaN,1779.02,36.0,Active,NaN,NaN
3,1c597777-8cb8-4f1b-b485-bf79ae4206a2,FoneYam,FoneYam (Pty) Ltd,https://foneyam.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN
4,a30c11ef-2153-425b-a3ce-9748d88cd35c,Standard Bank Connect,Standard Bank Connect (MVNO),https://connect.standardbank.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN


In [ ]:
df.head()

,id,name,legal_name,website,headquarters_city,founded_year,employee_count,annual_revenue_usd,financing_volume_usd,market_share_percentage,business_model,financing_types,credit_requirements,avg_approval_rate,avg_interest_rate,max_financing_amount_usd,avg_term_months,status,created_at,updated_at
0,338f1999-1a3c-43e2-8d31-f6076191fecb,Cellucity,Cellucity (Pty) Ltd,https://cellucity.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5c0101bc-e330-4cbd-968f-712e83458e6d,PayJoy South Africa,PayJoy South Africa (Pty) Ltd,https://www.payjoy.com/za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,88487866-2191-406c-b0b7-5a07d81def4b,Vodacom South Africa,Vodacom (Pty) Ltd,https://www.vodacom.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1c597777-8cb8-4f1b-b485-bf79ae4206a2,FoneYam,FoneYam (Pty) Ltd,https://foneyam.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a30c11ef-2153-425b-a3ce-9748d88cd35c,Standard Bank Connect,Standard Bank Connect (MVNO),https://connect.standardbank.co.za,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
